# LSTM for Diplomacy Deception Detection

This notebook implements a Bi-Directional LSTM model to detect deception in Diplomacy game messages. 
Unlike previous approaches using TF-IDF, this model uses raw text sequences to capture context and word order.

## Steps:
1. Load Data (Final enriched parquet files)
2. Text Preprocessing (Tokenization, Padding)
3. Model Definition (Embedding -> Bi-LSTM -> Dense)
4. Training with Class Weights (to handle imbalance)
5. Evaluation

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sns

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Define Paths
base_path = os.path.dirname(os.path.dirname(os.getcwd()))
data_path = os.path.join(base_path, "data", "processed", "diplomacy")

print(f"Data Path: {data_path}")

In [ ]:
# Load Data
train_df = pd.read_parquet(os.path.join(data_path, "train_final.parquet"))
val_df = pd.read_parquet(os.path.join(data_path, "val_final.parquet"))
test_df = pd.read_parquet(os.path.join(data_path, "test_final.parquet"))

print(f"Train shape: {train_df.shape}")
print(f"Val shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

# Check class distribution
print("\nClass Distribution (Train):")
print(train_df['target'].value_counts(normalize=True))

## Preprocessing
We use `message_text` (raw text) instead of `cleaned_text` to preserve stopwords and structure.

In [ ]:
# Parameters
VOCAB_SIZE = 10000
MAX_LEN = 150
EMBEDDING_DIM = 100
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOK = "<OOV>"

# Tokenization
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOK)
tokenizer.fit_on_texts(train_df['message_text'])

word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens.")

# Sequences
train_sequences = tokenizer.texts_to_sequences(train_df['message_text'])
val_sequences = tokenizer.texts_to_sequences(val_df['message_text'])
test_sequences = tokenizer.texts_to_sequences(test_df['message_text'])

# Padding
X_train = pad_sequences(train_sequences, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
X_val = pad_sequences(val_sequences, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
X_test = pad_sequences(test_sequences, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

y_train = train_df['target'].values
y_val = val_df['target'].values
y_test = test_df['target'].values

print(f"X_train shape: {X_train.shape}")

## Class Weights
Since the dataset is imbalanced, we calculate class weights to penalize the model more for missing the minority class (Deception).

In [ ]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights_dict}")

## Model Definition
We use a Bidirectional LSTM to capture context from both past and future words in the sequence.

In [ ]:
def build_model(vocab_size, embedding_dim, max_len):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN)
model.summary()

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5, verbose=1)

# Train
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

## Evaluation

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# Predictions
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Truth', 'Deception']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Truth', 'Deception'], yticklabels=['Truth', 'Deception'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Save Model
model.save(os.path.join(base_path, "models", "lstm_diplomacy.h5"))
print("Model saved.")